Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `???` or "YOUR ANSWER HERE".

---

# CSCI 3155: Assignment 9

Topics: 
- Type Systems
- Basics of Objects
- Traits

Readings: Notes posted on canvas.

In [1]:
// TEST HELPER
def passed(points: Int) {
    require(points >=0)
    if (points == 1) print(s"\n*** Tests Passed (1 point) ***\n")
    else print(s"\n*** Tests Passed ($points points) ***\n")
}

defined function passed

## Problem 1 (20 points)

In this problem, we will extend Lettuce language and its type system to references.  Here is a stripped down version of Lettuce we are going to use in this problem.

$$\begin{array}{rcl}
\mathbf{Expr} & \rightarrow & Const(\mathbf{Double}) \\
& | & Ident(\mathbf{Identifier})\\
& | & Plus(\mathbf{Expr}, \mathbf{Expr})\\
& | & Div(\mathbf{Expr}, \mathbf{Expr}) \\
& | & Geq(\mathbf{Expr}, \mathbf{Expr}) \\
& | & And(\mathbf{Expr}, \mathbf{Expr}) \\
& | & IfThenElse(\mathbf{Expr}, \mathbf{Expr}, \mathbf{Expr})\\
& | & Let(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}, \mathbf{Expr})\\
& | & FunDef(\mathbf{Identifier}, \color{red}{\mathbf{Type}}, \mathbf{Expr}) \\
& | & FunCall(\mathbf{Expr}, \mathbf{Expr}) \\
& | & \color{blue}{UnitConst} & \leftarrow \text{written as} \ () \\
& | & \color{blue}{NewRef(\mathbf{Expr})} \\
& | & \color{blue}{DeRef(\mathbf{Expr})} \\
& | & \color{blue}{AssignRef(\mathbf{Expr}, \mathbf{Expr})} & \leftarrow \text{written as} \ e_1 := e_2 \\
\end{array}$$

We extend the type system with a record type constructor and a new type as shown below.
$$\begin{array}{rcl}
\mathbf{Type} & \rightarrow & NumType & \leftarrow \text{written as } \ num \\
& | & BoolType & \leftarrow \text{written as } \ bool \\
& | & \color{red}{UnitType} & \leftarrow \text{written as } \ unit \\
& | & FunType(\mathbf{Type} , \mathbf{Type} ) & \leftarrow \text{written as } \ t_1 \Rightarrow t_2 \\
& | & \color{red}{RefType(\mathbf{Type})} & \leftarrow \text{written as } \ ref(t_1) \\
\end{array}$$




$$\newcommand\typeOf{\mathbf{typeOf}}$$
$$\newcommand\semRule[3]{\begin{array}{c} #1 \\ \hline #2 \\ \end{array}\;(\text{#3}) }$$
Recall from notes on "Types and Type Checking" that $\typeOf(\texttt{e}, \alpha)$ is the type of an 
expression $\texttt{e}$ under type environment $\alpha$. The type environment maps identifiers in the current scope to  their annotated types.

Let us write a rule for NewRef.

$$\semRule{ \typeOf(\texttt{e}, \alpha) = t,\; t \not= \mathbf{typeerror} }{\typeOf(\texttt{NewRef(e)}, \alpha) = ref(t) }{newref-ok}$$

It says that if $\texttt{e}$ receives type $t$ under type environment $\alpha$ and it is not a type error, then $\texttt{NewRef(e)}$ must receive the type $ref(t)$ under $\alpha$.

**Note** Complete the missing rules below on your own. We will not be grading them but you will need your results to finish the coding of the type checker.

 Complete the missing terms for the rule for `UnitConst` OK rule.
$$\semRule{}{\typeOf((), \alpha) = \color{red}{???_1}}{unitconst-ok}$$


 Complete the missing terms for the rule for `DeRef` OK rule.
$$\semRule{\typeOf(\texttt{e}, \alpha) = ref(t)}{\typeOf(\texttt{DeRef(e)}, \alpha) = \color{red}{???_2}}{deref-ok}$$

Complete the missing terms for the rule for `DeRef` error.
$$\semRule{\typeOf(\texttt{e}, \alpha) = t,\ t \not= \color{red}{???_3} }{\typeOf(\texttt{DeRef(e)}, \alpha) = \mathbf{typeerror}}{deref-nok}$$

The rules for `Deref(e)` must say that:
  - `e` must have type of the form `ref(t)` for some `t`.
  -  If yes then `Deref(e)` will then have the type `t`, otherwise `Deref(e)` will lead to `typeerror`.
 

We will change the semantics of `AssignRef` from class to have type `UnitType` instead of the type 
of expression `e2`. 
$$\semRule{\typeOf(\texttt{e1}, \alpha) = \color{red}{???_4}, \typeOf(\texttt{e2}, \alpha) = t}{\typeOf(\texttt{AssignRef(e1, e2)}, \alpha) =   unit}{assignref-ok}$$
The rule for `AssignRef(e1, e2)` must say that 
  - `e1` should be an expression of type `Ref(t)` for some `t`
  - `e2` should be an expression of that type `t`.
  - The entire expression `AssignRef(e1, e2)` gets the type `UnitType`.

We have the inductive definitions of the programs and the types below.

???1: UnitType
???2: Def(t) aka return t
???3: ref(t)
???4: t1, t1 == RefType(1)

In [2]:
sealed trait Type
case object NumType extends Type
case object BoolType extends Type
case class FunType(t1: Type, t2: Type) extends Type
case object UnitType extends Type
case class RefType(t: Type) extends Type


sealed trait Program
sealed trait Expr
case class Const(f: Double) extends Expr
case class Ident(s: String) extends Expr
case class Plus(e1: Expr, e2: Expr) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Geq(e1: Expr, e2: Expr) extends Expr
case class IfThenElse(e1: Expr, e2: Expr, e3: Expr) extends Expr
case class Let(x: String, xType: Type, e1: Expr, e2: Expr) extends Expr
case class FunDef(id: String, idType: Type, e: Expr) extends Expr
case class FunCall(calledFun: Expr, argExpr: Expr) extends Expr
case class NewRef(e: Expr) extends Expr
case class DeRef(e: Expr) extends Expr
case class AssignRef(e1: Expr, e2: Expr) extends Expr
case object UnitConst extends Expr
case class TopLevel(e: Expr) extends Program

def typeEquals(t1: Type, t2: Type): Boolean = t1 == t2
case class TypeErrorException(s: String) extends Exception


defined trait Type
defined object NumType
defined object BoolType
defined class FunType
defined object UnitType
defined class RefType
defined trait Program
defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Geq
defined class IfThenElse
defined class Let
defined class FunDef
defined class FunCall
defined class NewRef
defined class DeRef
defined class AssignRef
defined object UnitConst
defined class TopLevel
defined function typeEquals
defined class TypeErrorException

Complete the missing portions of the type checker below. Rather than have a explicit typeerror type, please throw the exception `TypeErrorException` with an appropriate error message whenever a erroneous type is encountered.

In [3]:
def typeOf(e: Expr, alpha: Map[String, Type]): Type = {
    def checkType(opName: String, e1: Expr, t1: Type, e2: Expr, t2: Type, resType: Type): Type = {
        val t1hat = typeOf(e1, alpha)
        if (! typeEquals(t1hat, t1)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t1, obtained $t1hat")
        }
        
        val t2hat = typeOf(e2, alpha)
        if (! typeEquals(t2hat, t2)){
            throw new TypeErrorException(s"Type mismatch in arithmetic/comparison/bool op $opName, Expected type $t2, obtained $t2hat")
        }
        
        resType
    }
    
    e match {
        case Const(f) => NumType
        case Ident(s) => {if (alpha contains s)
                             alpha(s)
                          else 
                             throw TypeErrorException(s"Unknown identifier $s")}
        case Plus(e1, e2) =>  checkType("Plus", e1,  NumType, e2, NumType, NumType)
        case Minus(e1, e2) => checkType("Minus",e1,  NumType, e2, NumType, NumType)
        case Geq(e1, e2) => checkType("Geq", e1,  NumType, e2, NumType, BoolType)
        case IfThenElse(e, e1, e2) => {
            val t = typeOf(e, alpha)
            if (t == BoolType){
                val t1 = typeOf(e1, alpha)
                val t2 = typeOf(e2, alpha)
                if (typeEquals(t1, t2))
                    t1
                else 
                    throw TypeErrorException(s"If then else returns unequal types $t1 and $t2")
            } else {
                throw TypeErrorException(s"If then else condition expression not boolean $t")
            }
        }

        case Let(x, t, e1, e2) => {
            val t1 = typeOf(e1, alpha)
            if (typeEquals(t1, t)){
                val newAlpha = alpha + (x -> t)
                typeOf(e2, newAlpha)
            } else {
                throw TypeErrorException(s"Let binding has type $t whereas it is bound to expression of type $t1")
            }
        }

        case FunDef(x, t1, e) => {
            val newAlpha = alpha + (x -> t1)
            val t2 = typeOf(e, newAlpha)
            FunType(t1, t2)
        }

        case FunCall(e1, e2) => {
            val ftype = typeOf(e1, alpha)
            ftype match {
                case FunType(t1, t2) => {
                    val argType = typeOf(e2, alpha)
                    if (typeEquals(argType, t1)){
                        t2
                    } else {
                        throw TypeErrorException(s"Call to function with incompatible argument type. Expected $t1, obtained $argType")
                    }
                }
                case _ => { throw TypeErrorException(s"Call to function but with a non function type $ftype")}

            }
        }

        case NewRef(e) => {
            // YOUR CODE HERE
           val t = typeOf(e, alpha)
           RefType(t)
            
        }
        
        case AssignRef(e1, e2) => {
            // YOUR CODE HERE
            val t1 = typeOf(e1, alpha)
            val t2 = typeOf(e2, alpha)
            
        t1 match {
            case RefType(t) => {
                if (typeEquals(t, t2)) {
                    UnitType
                } else {
                    throw TypeErrorException(s"Type mismatch in assignment. Expected type $t, obtained $t2")
                }
            }
            case _ => {
                throw TypeErrorException(s"Invalid type for assignment. Expected RefType, obtained $t1")
            }
        }
           
            
        }
        
        case DeRef(e) => {
            // YOUR CODE HERE
           val t = typeOf(e, alpha)
            t match {
                case RefType(tInner) => tInner
                case _ => throw TypeErrorException(s"Dereferencing a non-reference type $t")
            }
        }
        
    }
}

def typeOfProgram(p: Program) = p match {
    case TopLevel(e) => {
            val t = typeOf(e, Map())
            println(s"Program type computed successfully as $t")
            t
    }
}

defined function typeOf
defined function typeOfProgram

In [4]:
//BEGIN TEST

/* 
let x : ref(num) = NewRef(10 ) in 
   let dummy: unit = AssignRef(x, 30) in 
       DeRef(x)
       */

val p1 = Let("x", RefType(NumType), NewRef(Const(10)), Let("dummy", UnitType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t1 = typeOfProgram(TopLevel(p1))
assert(t1 == NumType, "Test 1 failed: answer should be NumType")
passed(5)
//END TEST

Program type computed successfully as NumType

*** Tests Passed (5 points) ***


p1: Let = Let(
  x = "x",
  xType = RefType(t = NumType),
  e1 = NewRef(e = Const(f = 10.0)),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = Ident(s = "x"), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t1: Type = NumType

In [5]:
//BEGIN TEST
/* 
let x : ref(num) = NewRef(function(z: num) z + 10) in 
   let dummy: unit = AssignRef(x, 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p2 = Let("x", RefType(NumType), NewRef(fdef), Let("dummy", UnitType, AssignRef(Ident("x"), Const(30) ), DeRef(Ident("x"))) )
val t2 = try {
   typeOfProgram(TopLevel(p2))
   assert(false, "The program should not receive a type")
} catch {
    case TypeErrorException(msg) => s"OK -- caught a type error exception: $msg"
    case e => print(e); assert(false, "Please throw TypeErrorException(message) when a type failure occurs")
}
passed(5)
//END TEST


*** Tests Passed (5 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
)
p2: Let = Let(
  x = "x",
  xType = RefType(t = NumType),
  e1 = NewRef(
    e = FunDef(
      id = "z",
      idType = NumType,
      e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
    )
  ),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = Ident(s = "x"), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t2: Any = "OK -- caught a type error exception: Let binding has type RefType(NumType) whereas it is bound to expression of type RefType(FunType(NumType,NumType))"

In [6]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: unit = AssignRef(NewRef(35), 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p4 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", UnitType, AssignRef(NewRef(Const(35)), Const(30) ), DeRef(Ident("x"))) )
val t4 =  typeOfProgram(TopLevel(p4))
assert(t4 == FunType(NumType, NumType), "Test failed")
passed(5)
//END TEST

Program type computed successfully as FunType(NumType,NumType)

*** Tests Passed (5 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
)
p4: Let = Let(
  x = "x",
  xType = RefType(t = FunType(t1 = NumType, t2 = NumType)),
  e1 = NewRef(
    e = FunDef(
      id = "z",
      idType = NumType,
      e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
    )
  ),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = NewRef(e = Const(f = 35.0)), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t4: Type = FunType(t1 = NumType, t2 = NumType)

In [7]:
//BEGIN TEST
/* 
let x : ref(num => num) = NewRef(function(z: num) z + 10) in 
   let dummy: unit = AssignRef(NewRef(35), 30) in 
       DeRef(x)
       */
val fdef = FunDef("z", NumType, Plus(Ident("z"), Const(10)))
val p4 = Let("x", RefType(FunType(NumType, NumType)), NewRef(fdef), Let("dummy", UnitType, AssignRef(NewRef(Const(35)), Const(30) ), DeRef(Ident("x"))) )
val t4 =  typeOfProgram(TopLevel(p4))
assert(t4 == FunType(NumType, NumType), "Test failed")
passed(5)
//END TEST

Program type computed successfully as FunType(NumType,NumType)

*** Tests Passed (5 points) ***


fdef: FunDef = FunDef(
  id = "z",
  idType = NumType,
  e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
)
p4: Let = Let(
  x = "x",
  xType = RefType(t = FunType(t1 = NumType, t2 = NumType)),
  e1 = NewRef(
    e = FunDef(
      id = "z",
      idType = NumType,
      e = Plus(e1 = Ident(s = "z"), e2 = Const(f = 10.0))
    )
  ),
  e2 = Let(
    x = "dummy",
    xType = UnitType,
    e1 = AssignRef(e1 = NewRef(e = Const(f = 35.0)), e2 = Const(f = 30.0)),
    e2 = DeRef(e = Ident(s = "x"))
  )
)
t4: Type = FunType(t1 = NumType, t2 = NumType)

## Problem 2 (20 points)

Suppose we wish to define an abstract class called `Shape` that is a three-dimensional geometric shape. It needs to have methods such as  `getCenter`,  and `translate`.

  - `getCenter` : Gets the center of the shape. For a polyhedron the center is the "average" of the coordinates of its vertices. For a sphere/ellipsoid, the center is part of the description of the object (see below).
  - `translate`: Translates the shape along some given values of `xShift`, `yShift`, and `zShift`.


We will define a trait called `WithCorners` that define shapes with corners. This trait will implement a function called `getVertices`.

There are many classes that inherit from `Shape` including `Polyhedron` and `Ellipsoid`. The classes `Pyramid` and `Cuboid` inherit from `Polyhedron` and `Sphere` inherits from `Ellipsoid`. 

Also the class `Polyhedron` and all its derived classes must mixin the trait `WithCorners` but the class `Ellipsoid` and its derived classes should not.

- class `Pyramid(x0: (Double, Double, Double), x1: (Double, Double, Double), x2: (Double, Double, Double), x3: (Double, Double, Double))` must have four class parameters representing the four corners of the pyramid.
  - It must inherit from `Polyhedron`.
  - It must mixin the trait `WithCorners`.
  - It must override the `translate` method so that `translate` called on a `Pyramid` returns an instance of a `Pyramid`.
  
- class `Cuboid(lowerLeft: (Double, Double, Double), length: Double, width: Double, height:Double)` must have the class parameters representing the lower left coordinate, length,  width, and height.
  - It must inherit from `Polyheron`.
  - It must mixin the trait `WithCorners`.
  - It must override the `translate` method so that `translate` called on a `Cuboid` returns an instance of a `Cuboid`.
  
- class `Sphere(center: (Double, Double, Double), rad: Double)` must have its center coordinate and radius as class paramters.
  - It must inherit from Ellipsoid.
  - It must override the `translate` method so that `translate` called on a `Sphere` returns an instance of a `Sphere`.





### Part A
Complete the definitions below so that your code compiles.

**Restrictions** No loops, mutables (var) or recursion. Please use functors `map`, `foldLeft` etc.. when possible.


In [8]:
abstract class Shape {
    def getCenter:(Double, Double, Double)
    def translate(xShift: Double, yShift: Double, zShift: Double): Shape
}

trait WithCorners {
    def getVertices: List[(Double, Double, Double)]
}

class Ellipsoid(val center: (Double, Double, Double), val axisLengths: (Double, Double, Double)) extends Shape {
    //TODO: Finish the methods that need to be implemented.
    // YOUR CODE HERE
    def getCenter:(Double, Double, Double) = center
    def translate(xShift: Double, yShift: Double, zShift: Double):Ellipsoid = {
        new Ellipsoid((center._1 + xShift, center._2+ yShift, center._3 + zShift), axisLengths) 
    }
} 

class Polyhedron(val listOfVerts: List[(Double, Double, Double)]) extends Shape with WithCorners {
     assert(listOfVerts.length >= 1)
    // TODO: Finish the methods that need to be implemented.
    // YOUR CODE HERE
    def getVertices: List[(Double, Double, Double)] = listOfVerts
    def getCenter:(Double, Double, Double) = {
        val n = listOfVerts.length
        val (xSum, ySum, zSum) = listOfVerts.foldLeft((0.0, 0.0, 0.0)) {
            case ((xAcc, yAcc, zAcc), (x, y, z)) => (xAcc + x, yAcc + y, zAcc + z)
        }
    (xSum/n, ySum/n, zSum/n)
    }
    def translate(xShift: Double, yShift: Double, zShift: Double): Polyhedron = {
        val translated = new Polyhedron(listOfVerts.map(a => (a._1 + xShift, a._2 + yShift, a._3 + zShift)))
        translated
    }
}

//TODO: Complete definitions of pyramid, cuboid and sphere classes as specified.
// YOUR CODE HERE
class Pyramid(x0: (Double, Double, Double), x1: (Double, Double, Double), x2: (Double, Double, Double), x3: (Double, Double, Double)) extends Polyhedron(List(x0, x1, x2, x3)) with WithCorners 
{
  override def getVertices: List[(Double, Double, Double)] = listOfVerts
  override def translate(x: Double, y: Double, z: Double): Pyramid = {
    val translatedVertices = listOfVerts.map(v => (v._1 + x, v._2 + y, v._3 + z))
    new Pyramid(translatedVertices(0), translatedVertices(1), translatedVertices(2), translatedVertices(3))
  }
}
class Cuboid(lowerLeft: (Double, Double, Double), length: Double, width: Double, height: Double) extends Polyhedron(List(
    lowerLeft,
    (lowerLeft._1 + length, lowerLeft._2, lowerLeft._3),
    (lowerLeft._1 + length, lowerLeft._2 + width, lowerLeft._3),
    (lowerLeft._1, lowerLeft._2 + width, lowerLeft._3),
    (lowerLeft._1, lowerLeft._2, lowerLeft._3 + height),
    (lowerLeft._1 + length, lowerLeft._2, lowerLeft._3 + height),
    (lowerLeft._1 + length, lowerLeft._2 + width, lowerLeft._3 + height),
    (lowerLeft._1, lowerLeft._2 + width, lowerLeft._3 + height)
    )) with WithCorners
{
    override def getVertices: List[(Double, Double, Double)] = listOfVerts
    override def translate(x:Double, y:Double, z:Double): Cuboid =  {
       val left = (lowerLeft._1 + x, lowerLeft._2 + y, lowerLeft._3 + z)
        new Cuboid(left, length, width, height)
    }

}
class Sphere(override val center: (Double, Double, Double), val rad: Double) extends Ellipsoid(center, (rad, rad, rad)) {
    override def getCenter: (Double, Double, Double) = center
    override def translate(x: Double, y: Double, z: Double): Sphere = {
        new Sphere((center._1 + x, center._2 + y, center._3 + z), rad)
    }
}



defined class Shape
defined trait WithCorners
defined class Ellipsoid
defined class Polyhedron
defined class Pyramid
defined class Cuboid
defined class Sphere

In [9]:
val el1 = new Ellipsoid((5.0, -3.5, 0.0), (2.1, 1.3, 2.0))
val el2 = el1.translate(3,4,3)
assert (el2.center == (8.0, 0.5, 3.0), "Test failed: After translation, the center must be (8,0.5,3.0)")

val pl1 = new Polyhedron(List((0,0,0), (2,1,2)))
assert (pl1.getCenter == (1.0, 0.5, 1.0), "center must be (1,0.5)")

passed(5)


*** Tests Passed (5 points) ***


el1: Ellipsoid = ammonite.$sess.cmd7$Helper$Ellipsoid@25ecdfbd
el2: Ellipsoid = ammonite.$sess.cmd7$Helper$Ellipsoid@3c4a747a
pl1: Polyhedron = ammonite.$sess.cmd7$Helper$Polyhedron@4c79e3cd

In [10]:
val pyr = new Pyramid( (0,0, 0), (2,0,0), (1,1,0), (1.5, 1.5, 3))
val pyr2 = pyr.translate(1,1,1)
assert (pyr2.isInstanceOf[Pyramid])
print(pyr2.getCenter)
assert(pyr2.getCenter._1 == 2.125)
assert(pyr2.getCenter._3 == 1.75)
passed(5)

(2.125,1.625,1.75)
*** Tests Passed (5 points) ***


pyr: Pyramid = ammonite.$sess.cmd7$Helper$Pyramid@526ec1da
pyr2: Pyramid = ammonite.$sess.cmd7$Helper$Pyramid@79889cd

In [11]:
val cub = new Cuboid((-1,-1,-1), 2, 2, 2)
val listOfVerts = cub.getVertices
assert(listOfVerts.length == 8)
print(cub.getCenter)
assert (cub.getCenter == (0,0,0))
val cub2: Cuboid = cub.translate(1,1,1)
print(cub2.getCenter)
assert(cub2.getCenter == (1,1,1))
passed(5)

(0.0,0.0,0.0)(1.0,1.0,1.0)
*** Tests Passed (5 points) ***


cub: Cuboid = ammonite.$sess.cmd7$Helper$Cuboid@5b8a5ece
listOfVerts: List[(Double, Double, Double)] = List(
  (-1.0, -1.0, -1.0),
  (1.0, -1.0, -1.0),
  (1.0, 1.0, -1.0),
  (-1.0, 1.0, -1.0),
  (-1.0, -1.0, 1.0),
  (1.0, -1.0, 1.0),
  (1.0, 1.0, 1.0),
  (-1.0, 1.0, 1.0)
)
cub2: Cuboid = ammonite.$sess.cmd7$Helper$Cuboid@60721ee6

In [12]:
val spr = new Sphere((1,1,1), 3)
val new_spr: Sphere = spr.translate(2,2,2)
assert (new_spr.center == (3,3,3))
assert (new_spr.rad == 3)
passed(5)


*** Tests Passed (5 points) ***


spr: Sphere = ammonite.$sess.cmd7$Helper$Sphere@3f22e9cd
new_spr: Sphere = ammonite.$sess.cmd7$Helper$Sphere@6ead4288